In [1]:
import tensorflow
physical_devices = tensorflow.config.list_physical_devices('GPU')
tensorflow.config.experimental.set_memory_growth(physical_devices[0], True)

## Dogs vs Cats Image Classification - VGG16

In [4]:
print("Num GPUs Available: ", len(tensorflow.config.list_physical_devices('GPU')))
classes = 2
img_width, img_height, img_channels = 224, 224, 3
input_shape = (img_width, img_height, img_channels)

Num GPUs Available:  1


In [5]:
from utilities import get_train_val_iterator
train_iterator, val_iterator = get_train_val_iterator( 
    input_shape, 
    "datasets//dogs-vs-cats//train",
    "datasets//dogs-vs-cats//val",
    batch_size = 64
)

Found 18697 images belonging to 2 classes.
Found 6303 images belonging to 2 classes.


### VGG-16 Sequential API 

In [6]:
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential( name="VGG16_basic" )

# Convolution Pooling Block 1
model.add( Conv2D( 64, kernel_size=(3,3), strides=1, padding="SAME", activation="relu", 
                  input_shape=(224, 224, 3 ) ))
model.add( Conv2D( 64, kernel_size=(3,3), strides=1, padding="SAME", activation="relu" ))
model.add( MaxPooling2D( pool_size=(2,2), strides=(2,2) ) )

# Convolution Pooling Block 2
model.add( Conv2D(128, kernel_size=(3,3), padding="SAME", activation="relu" ) )
model.add( Conv2D(128, kernel_size=(3,3), padding="SAME", activation="relu" ) )
model.add( MaxPooling2D( pool_size=(2,2), strides=(2,2) ) )
          
# Convolution Pooling Block 3
model.add( Conv2D(256, kernel_size=(3,3), padding="SAME", activation="relu" ) )
model.add( Conv2D(256, kernel_size=(3,3), padding="SAME", activation="relu" ) )
model.add( Conv2D(256, kernel_size=(3,3), padding="SAME", activation="relu" ) )
model.add( MaxPooling2D( pool_size=(2,2), strides=(2,2) ) )

# Convolution Pooling Block 4
model.add( Conv2D(512, kernel_size=(3,3), padding="SAME", activation="relu" ) )
model.add( Conv2D(512, kernel_size=(3,3), padding="SAME", activation="relu" ) )
model.add( Conv2D(512, kernel_size=(3,3), padding="SAME", activation="relu" ) )
model.add( MaxPooling2D( pool_size=(2,2), strides=(2,2) ) )

# Convolution Pooling Block 5
model.add( Conv2D(512, kernel_size=(3,3), padding="SAME", activation="relu" ) )
model.add( Conv2D(512, kernel_size=(3,3), padding="SAME", activation="relu" ) )
model.add( Conv2D(512, kernel_size=(3,3), padding="SAME", activation="relu" ) )
model.add( MaxPooling2D( pool_size=(2,2), strides=(2,2) ) )         

# Flatten
model.add( Flatten() )

# Dense Layers
model.add( Dense( 4096, activation='relu' ) )
model.add( Dense( 4096, activation='relu' ) )
model.add( Dense( classes, activation='softmax' ) )          

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "VGG16_basic"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 112, 112, 128)     147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 56, 56, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 56, 56, 256)       

In [7]:
earlyStopping = tensorflow.keras.callbacks.EarlyStopping(
                    monitor='val_loss', patience=5, verbose=1 )

history = model.fit_generator(
    generator = train_iterator, 
    steps_per_epoch = len(train_iterator), 
    validation_data = val_iterator, 
    validation_steps = len(val_iterator), 
    epochs = 500,
    callbacks = [ earlyStopping ]
)

Instructions for updating:
Please use Model.fit, which supports generators.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 293 steps, validate for 99 steps
Epoch 1/500
  1/293 [..............................] - ETA: 57:44WARNING:tensorflow:Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: 


ResourceExhaustedError:  OOM when allocating tensor with shape[64,64,224,224] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node VGG16_basic/conv2d/Conv2D (defined at <ipython-input-7-6811c9b68848>:10) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_distributed_function_1956]

Function call stack:
distributed_function


In [ ]:
from utilities import plot_training_history
plot_training_history(history)

In [ ]:
# Save model
save_path = "saved_models\\" + model.name + ".h5"
model.save( save_path )